# Overview
Our study-test version so far has an "imbalaced" order (20 targets, 44 lures, 20 foils) and an even-ish representation of bins 1-5.  Here, we want to skew that distribution of lure bins.  We're going to have it do 5,6,11,11,11

Our continuous version has been focused on 3 bins (e.g., 2-4) and we're going to cover the range but skew it as well

In [9]:
# Lifted originally from CreateImbalancedOrders.py, which makes study-test format .js order files
""" 3/22/2021

For this second iteration of 'how to compress the MST into the cMST', we're taking a 
different approach.  It seems the ON vs. OSN was likely the problem before.  But, if
we're going to be on OSN (and maybe even if not), we really want to make sure we
have as many L trials as possible.

So, here, we're going to try:
Study 64 items
  - 20 go on to be repeated
  - 44 go on to be lures
Test 44 items
  - 20 repeats
  - 44 lures
  - 20 novel foils

The code here lifts heavily from our orignal MST_PsychoPy.py but makes the output
in the .js format directly (rather than needing ConvertOrdersCSVtoJKS.py) """

# 7/12/22 (CELS): Revise this to have our skewed distribution of lure bins (5,6,11,11,11)


import os, csv
import numpy as np
import glob

nrepeats=20
nlures=44
nfoils=20

NImagePairs = 192

print(os.getcwd())
#basepath=os.path.join('D:',os.sep,'dev','jatos','study_assets_root','MST_STvsCont')
basepath=os.getcwd()
os.chdir(basepath)


def check_files(SetName,validate_images=False):
    """ 
    SetName should be something like "C" or "1"
    Checks to make sure there are the right #of images in the image directory
    Loads the lure bin ratings into the global set_bins list and returns this
    """

    #print(SetName)
    #print(P_N_STIM_PER_LIST)
    
    bins = []  # Clear out any existing items in the bin list

    # Load the bin file
    with open("Set"+str(SetName)+" bins.txt","r") as bin_file:
        reader=csv.reader(bin_file,delimiter='\t')
        for row in reader:
            if int(row[0]) > NImagePairs:
                raise ValueError('Stimulus number ({0}) too large - not in 1-192 in binfile'.format(row[0]))
            if int(row[0]) < 1:
                raise ValueError('Stimulus number ({0}) too small - not in 1-192 in binfile'.format(row[0]))
            bins.append(int(row[1]))
    if len(bins) != NImagePairs:
        raise ValueError('Did not read correct number of bins in binfile')
    
    # Check the stimulus directory
    if validate_images:
        img_list=glob.glob("Set" +str(SetName) + '_rs' + os.sep + '*.jpg')
        if len(img_list) < (2*NImagePairs):
            raise ValueError('Not enough files in stimulus directory {0} -- {1} vs {2}'.format("Set " +str(SetName) + os.sep + '*.jpg',len(img_list),2*NImagePairs))
        for i in range(1,NImagePairs+1):
            if not os.path.isfile("Set" +str(SetName) + '_rs' + os.sep + '{0:03}'.format(i) + 'a.jpg'):
                raise ValueError('Cannot find: ' + "Set " +str(SetName) + os.sep + '{0:03}'.format(i) + 'a.jpg')
            if not os.path.isfile("Set" +str(SetName) + '_rs' + os.sep + '{0:03}'.format(i) + 'b.jpg'):
                raise ValueError('Cannot find: ' + "Set " +str(SetName) + os.sep + '{0:03}'.format(i) + 'b.jpg')
    return bins


def setup_list_permuted(set_bins):
    """
    set_bins = list of bin values for each of the 192 stimuli -- set specific
    
    Assumes check_files() has been run so we have the bin numbers for each stimulus

    Assumes the ntargets,nlures, and nfoils set globally
    
    Not setup to do sublists at all -- 
    
    Returns lists with the image numbers for each stimulus type (study, repeat...)
    in the to-be-used permuted order with the to-be-used list size

    7/12/22: This got hacked / generalized considerably

    """

    
    if len(set_bins) != NImagePairs:
        raise ValueError('Set bin length is not the same as the stimulus set length ({0})'.format(NImagePairs))

    if (nrepeats > 64) or (nlures > 64) or (nfoils > 64):
        raise ValueError('Not setup to have any stim type >64 at the moment')
        
    # Figure the image numbers for the lure bins
    lure1=np.where(set_bins == 1)[0] + 1
    lure2=np.where(set_bins == 2)[0] + 1
    lure3=np.where(set_bins == 3)[0] + 1
    lure4=np.where(set_bins == 4)[0] + 1
    lure5=np.where(set_bins == 5)[0] + 1
    
    # Permute these
    lure1 = np.random.permutation(lure1)
    lure2 = np.random.permutation(lure2)
    lure3 = np.random.permutation(lure3)
    lure4 = np.random.permutation(lure4)
    lure5 = np.random.permutation(lure5)

    lures=np.empty(nlures,dtype=int)
    #nonlures=np.empty(NImagePairs-nlures,dtype=int)

    # Build up my list of lures and nonlures
    bincount=np.array([5,6,11,11,11])
    binind=bincount.cumsum()
    lures[0:binind[0]]=lure1[0:bincount[0]]  # 5 of these
    nonlures=lure1[bincount[0]:]
    lures[binind[0]:binind[1]]=lure2[0:bincount[1]]
    nonlures=np.hstack((nonlures,lure2[bincount[1]:]))
    lures[binind[1]:binind[2]]=lure3[0:bincount[2]]
    nonlures=np.hstack((nonlures,lure3[bincount[2]:]))
    lures[binind[2]:binind[3]]=lure4[0:bincount[3]]
    nonlures=np.hstack((nonlures,lure4[bincount[3]:]))
    lures[binind[3]:binind[4]]=lure5[0:bincount[4]]
    nonlures=np.hstack((nonlures,lure5[bincount[4]:]))
    
    # Randomize the lures and non-lures and split into the lengths we need
    lurestim=np.random.permutation(lures)
    nonlures=np.random.permutation(nonlures)
    foilstim = nonlures[0:nfoils]
    repeatstim = nonlures[nfoils:nfoils+nrepeats]
              
    return (repeatstim,lurestim,foilstim)
    

def create_order(p_set, repeatstim, lurestim, foilstim):
    """
    p_set = Set we're using (e.g., '1', or 'C')
    repeatstim,lurestim,foilstim: Lists (np.arrays actually) created by setup_list_permuted
    
    Returns lists with the filenames and conditions for each trial in both the
    study and test phases
        
    """
    #n_per = len(repeatstim)
    # Do the study phase - easy as we already have the list and it's still
    # setup with the first half being the to-be-repeated and the 2nd half
    # being the to-be-lured stimuli
    # Create the study list - note the first half are the to-be-repeated and
    # the second half are the to-be-lured at this point
    study_stim = np.concatenate((repeatstim,lurestim)) 
    # Set up a condition list to reflect this
    study_cond=[]
    study_cond = ['SR']*nrepeats
    study_cond[nrepeats:(nrepeats+nlures)] = ['SL']*nlures
    study_cond=np.array(study_cond)  # Make it an np-array so we can index easiy
    order = np.random.permutation(nrepeats+nlures)
    study_cond = list(study_cond[order])
    study_list=[]
    for i in range(nrepeats+nlures):
        study_list.append('Set{0}_rs{1}{2:03}a.jpg'.format(p_set, '/', study_stim[order[i]]))
    

    # Do the test phase in a similar way 
    test_stim = np.concatenate((repeatstim,lurestim,foilstim)) 
    test_cond=[]
    test_cond=['TR']*nrepeats
    test_cond[nrepeats:(nrepeats+nlures)] = ['TL']*nlures
    test_cond[(nrepeats+nlures):] = ['TF']*nfoils
    test_cond=np.array(test_cond)  # Make it an np-array so we can index easiy
    order = np.random.permutation(nrepeats+nlures+nfoils)
    test_cond = list(test_cond[order])
    test_list=[]
    for i in range(nrepeats+nlures+nfoils):
        if test_cond[i] == 'TL':  # Use the 'b' version only for the lures
            test_list.append('Set{0}_rs{1}{2:03}b.jpg'.format(p_set,'/',test_stim[order[i]]))
        else:
            test_list.append('Set{0}_rs{1}{2:03}a.jpg'.format(p_set,'/',test_stim[order[i]]))
        
        
    return (study_list,study_cond,test_list,test_cond)


def write_jsfile(study_list,study_cond,test_list,test_cond,set_bins,prefix='MST_imbal2',stimset='1',order='1'):
    outfname=os.path.join(basepath,'jsOrders',prefix+'_s'+stimset+'_p1_o'+order+'.js')
    print('saving to ' + outfname)
    outfile=open(outfname,"w")
    outfile.write('var trial_stim=[\n')
    for i in range(len(study_cond)):
        outfile.write('  {' + "stim: '{0}', cond: '{1}'".format(study_list[i],study_cond[i]) + '}')
        if i < (len(study_cond)-1):
            outfile.write(',\n')
        else:
            outfile.write('\n')
    outfile.write(']\n')
    outfile.close()

    outfname=os.path.join(basepath,'jsOrders',prefix+'_s'+stimset+'_p2_o'+order+'.js')
    print('saving to ' + outfname)
    outfile=open(outfname,"w")
    outfile.write('var trial_stim=[\n')
    for i in range(len(test_cond)):
        stimnum=int(test_list[i][8:11]) # Should work given our fixed naming convention
        lbin=set_bins[stimnum-1] # images are 1-indexed
        if test_cond[i]=='TR':
            corr3='0'
            corr2='0'
        elif test_cond[i]=='TL':
            corr3='1'
            corr2='1'
        elif test_cond[i]=='TF':
            corr3='2'
            corr2='1'
        else:
            raise ValueError('Unknown test condition')
        outfile.write('  {' + "stim: '{0}', cond: '{1}', lbin: {2}, corr3: {3}, corr2: {4}".format(
                test_list[i],test_cond[i],lbin,corr3,corr2) + '}')
        if i < (len(test_cond)-1):
            outfile.write(',\n')
        else:
            outfile.write('\n')
    outfile.write(']\n')
    outfile.close()


def CreateOrderFile(prefix='MST_imbal2',stimset='1',order='1'):
    set_bins = np.array(check_files(stimset))

    # Figure out which stimuli will be shown in which conditions and order them
    (repeat_list, lure_list, foil_list) = setup_list_permuted(set_bins)
    
    (study_list,study_cond,test_list,test_cond) = create_order(stimset,repeat_list, lure_list, foil_list)
    
    write_jsfile(study_list,study_cond,test_list,test_cond,set_bins,prefix,stimset,order)


#CreateOrderFile(stimset='1')
#CreateOrderFile(stimset='2')
#CreateOrderFile(stimset='3')
#CreateOrderFile(stimset='4')
#CreateOrderFile(stimset='5')
#CreateOrderFile(stimset='6')
#CreateOrderFile(stimset='1',prefix='MST_imbalX')


c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc


In [11]:
#CreateOrderFile(stimset='1')
CreateOrderFile(stimset='2')
CreateOrderFile(stimset='3')
CreateOrderFile(stimset='4')
CreateOrderFile(stimset='5')
CreateOrderFile(stimset='6')

saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrders\MST_imbal1_s2_p1_o1.js
saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrders\MST_imbal1_s2_p2_o1.js
saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrders\MST_imbal1_s3_p1_o1.js
saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrders\MST_imbal1_s3_p2_o1.js
saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrders\MST_imbal1_s4_p1_o1.js
saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrders\MST_imbal1_s4_p2_o1.js
saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrders\MST_imbal1_s5_p1_o1.js
saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrders\MST_imbal1_s5_p2_o1.js
saving to c:\Users\craig\dev\jatos_win_java\study_assets_root\MST_vs_cMST_v2\misc\jsOrde

# Do the continuous version as well
A number of the above routines are the same for the continuous version of the task:
```
setup_list_permuted()
check_files()
```
and our globals can still hold, but we need the functions below


In [25]:
def load_and_decode_order(repeat_list,lure_list,foil_list,set_bins,
                          lag_set='Copt_4-30_orders',order=1,base_dir='.',
                          stim_set='1'):
    """
    Loads the order text file and decodes this into a list of image names, 
     conditions, lags, etc.
    
    lag_set: Directory name with the order files
    order: Which order file to use (numeric index)
    base_dir: Directory that holds the set of lag sets
    stim_set = Set we're using (e.g., '1', or 'C')
    repeat_list,lure_list,foil_list: Lists (np.arrays actually) created by setup_list_permuted
    

    In the order files files we have 2 columns:
        1st column is the stimulus type + number:
        Offset_1R = 0; % 1-100 1st of repeat pair
        Offset_2R = 100; % 101-200  2nd of repeat pair
        Offset_1L = 200; % 201-300 1st of lure pair
        Offset_2L = 300; % 301-400 2nd of lure pair
        Offset_Foil = 400; % 401+ Foil
        
        2nd column is the lag + 500 (-1 for 1st and foil)

    Returns:
        lists / arrays that are all N-trials long
        
        type_code: 0=1st of repeat
                   1=2nd of repeat
                   2=1st of lure
                   3=2nd of lure
                   4=foil
        ideal_resp: 0=old
                    1=similar
                    2=new
        lag: Lag for this item (-1=1st/foil, 0=adjacent, N=items between)
        fnames: Actual filename of image to be shown
    """
    fname=base_dir + os.sep + lag_set + os.sep + "order_{0}.txt".format(order)
    fdata=np.genfromtxt(fname,dtype=int,delimiter=',')
    
    lag = fdata[:,1]
    lag[lag != -1] = lag[lag != -1] - 500
    
    type_code = fdata[:,0]//100  #Note, this works b/c we loaded the data as ints
    
    stim_index = fdata[:,0]-100*type_code
    #print(stim_index)
    #set_bins = np.array(check_files(stim_set))
    #lbins=set_bins[stim_index-1]
    #print(set_bins,'len=',len(set_bins))
    #print(stim_index,'len=',len(stim_index))
    #print(lbins,'len=',len(lbins))
    lbins=np.zeros_like(stim_index)
    
    ideal_resp = np.zeros_like(stim_index)
    ideal_resp[type_code==4]=2
    ideal_resp[type_code==0]=2
    ideal_resp[type_code==2]=2
    ideal_resp[type_code==1]=0
    ideal_resp[type_code==3]=1
    
    fnames=[]
#    dirname='Set {0}{1}'.format(stim_set, os.sep)  # Get us to the directory
    dirname='Set {0}_rs/'.format(stim_set)  # Get us to the directory
    for i in range(len(type_code)):
        stimfile='UNKNOWN'
        if type_code[i]==0 or type_code[i]==1:
            stimfile='{0:03}a.jpg'.format(repeat_list[stim_index[i]-1])
            lbins[i]=set_bins[repeat_list[stim_index[i]-1]-1]
        elif type_code[i]==2:
            stimfile='{0:03}a.jpg'.format(lure_list[stim_index[i]-1])
            lbins[i]=set_bins[lure_list[stim_index[i]-1]-1]
        elif type_code[i]==3:
            stimfile='{0:03}b.jpg'.format(lure_list[stim_index[i]-1])
            lbins[i]=set_bins[lure_list[stim_index[i]-1]-1]
        elif type_code[i]==4:
            stimfile='{0:03}a.jpg'.format(foil_list[stim_index[i]-1])
            lbins[i]=set_bins[lure_list[stim_index[i]-1]-1]
        fnames.append(dirname+stimfile)
    
    return (type_code,ideal_resp,lag,fnames,lbins)

def CreateJSFile(lag_set='Copt_4-30_orders', stim_set='1', order=1, nruns=20,base_dir='.'):
    set_bins = np.array(check_files(stim_set))
    # Figure out which stimuli will be shown in which conditions and order them
    print('Ordering...')
    repeat_list, lure_list, foil_list = setup_list_permuted(set_bins)
    
    # Load up the order file and decode it, creating all the needed vectors
    type_code,ideal_resp,lag,fnames,lbins=load_and_decode_order(repeat_list,
            lure_list,foil_list,set_bins,lag_set=lag_set,
            order=order, stim_set=stim_set,base_dir=base_dir)
    for run in range(nruns):
        outname='jsOrders{0}{1}_{2}_{3}_{4}.js'.format(os.sep,lag_set.replace('_orders','2_orders'),stim_set,order,run+1) 
        print(outname)
        fp=open(outname,'w')
        fp.write('var trial_stim=[\n')
        for i in np.arange(len(type_code)):
            #lbin=set_bins[stimnum-1] # images are 1-indexed
            fp.write('  {' + "trial: {0}, image: '{1}', type: {2}, correct_resp: {3}, lag: {4}, lbin: {5}".format(
                    i,fnames[i],type_code[i],ideal_resp[i],lag[i],lbins[i]) +'}')
            if i < (len(type_code)-1):
                fp.write(',\n')
            else:
                fp.write('\n')
        fp.write(']\n')
        fp.close()

def ImbalCreate():
    for sset in np.arange(1,7):
        for order in np.arange(1,3):
            CreateJSFile(lag_set='cMST_Imbal_orders', stim_set=str(sset), order=order, nruns=1)

In [26]:
#CreateJSFile(lag_set='cMST_Imbal_orders', stim_set='1', order=1, nruns=1)
ImbalCreate()

Ordering...
jsOrders\cMST_Imbal2_orders_1_1_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_1_2_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_2_1_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_2_2_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_3_1_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_3_2_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_4_1_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_4_2_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_5_1_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_5_2_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_6_1_1.js
Ordering...
jsOrders\cMST_Imbal2_orders_6_2_1.js
